In [1]:
# SpaCy
import spacy
nlp = spacy.load('en_core_web_sm')

from spacy.lang.en import English
parser = English()

# nltk
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

# gensim
import gensim
from gensim import corpora

# others
import pickle
import random

[nltk_data] Downloading package wordnet to /Users/kautuk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kautuk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
en_stop = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    doc = nlp(text)
    noun_chunks = list(doc.noun_chunks) 
#     return lda_tokens + 10*[word.text for word in list(doc.noun_chunks) if not len(word.text.split(' '))>3]
    return [word.text for word in list(doc.noun_chunks) if not len(word.text.split(' '))>3]

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def prepare_text_for_lda(text):
    filtered_document = [w.lower() for w in text.split(' ')]
    text = ' '.join(filtered_document)
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [3]:
g1 = '''Glaciers are melting, sea levels are rising, cloud forests are dying, and wildlife is scrambling to keep pace. It has become clear that humans have caused most of the past century's warming by releasing heat-trapping gases as we power our modern lives. Called greenhouse gases, their levels are higher now than at any time in the last 800,000 years.
We often call the result global warming, but it is causing a set of changes to the Earth's climate, or long-term weather patterns, that varies from place to place. While many people think of global warming and climate change as synonyms, scientists use “climate change” when describing the complex shifts now affecting our planet’s weather and climate systems—in part because some areas actually get cooler in the short term.
Climate change encompasses not only rising average temperatures but also extreme weather events, shifting wildlife populations and habitats, rising seas, and a range of other impacts. All of those changes are emerging as humans continue to add heat-trapping greenhouse gases to the atmosphere, changing the rhythms of climate that all living things have come to rely on.
What will we do—what can we do—to slow this human-caused warming? How will we cope with the changes we've already set into motion? While we struggle to figure it all out, the fate of the Earth as we know it—coasts, forests, farms, and snow-capped mountains—hangs in the balance.
Understanding the greenhouse effect
The "greenhouse effect" is the warming that happens when certain gases in Earth's atmosphere trap heat. These gases let in light but keep heat from escaping, like the glass walls of a greenhouse, hence the name.
Sunlight shines onto the Earth's surface, where the energy is absorbed and then radiate back into the atmosphere as heat. In the atmosphere, greenhouse gas molecules trap some of the heat, and the rest escapes into space. The more greenhouse gases concentrate in the atmosphere, the more heat gets locked up in the molecules.
Scientists have known about the greenhouse effect since 1824, when Joseph Fourier calculated that the Earth would be much colder if it had no atmosphere. This natural greenhouse effect is what keeps the Earth's climate livable. Without it, the Earth's surface would be an average of about 60 degrees Fahrenheit (33 degrees Celsius) cooler.
Aren't temperature changes natural?
Human activity isn't the only factor that affects Earth's climate. Volcanic eruptions and variations in solar radiation from sunspots, solar wind, and the Earth's position relative to the sun also play a role. So do large-scale weather patterns such as El Niño.
But climate models that scientists use to monitor Earth’s temperatures take those factors into account. Changes in solar radiation levels as well as minute particles suspended in the

 atmosphere from volcanic eruptions, for example, have contributed only about two percent to the recent warming effect. The balance comes from greenhouse gases and other human-caused factors, such as land use change.
The short timescale of this recent warming is singular as well. Volcanic eruptions, for example, emit particles that temporarily cool the Earth's surface. But their effect lasts just a few years. Events like El Niño also work on fairly short and predictable cycles. On the other hand, the types of global temperature fluctuations that have contributed to ice ages occur on a cycle of hundreds of thousands of years.
For thousands of years now, emissions of greenhouse gases to the atmosphere have been balanced out by greenhouse gases that are naturally absorbed. As a result, greenhouse gas concentrations and temperatures have been fairly stable, which has allowed human civilization to flourish within a consistent climate.
Now, humans have increased the amount of carbon dioxide in the atmosphere by more than a third since the Industrial Revolution. Changes that have historically taken thousands of years are now happening over the course of decades.
Why does this matter?
The rapid rise in greenhouse gases is a problem because it’s changing the climate faster than some living things can adapt to. Also, a new and more unpredictable climate poses unique challenges to all life.
Historically, Earth's climate has regularly shifted between temperatures like those we see today and temperatures cold enough to cover much of North America and Europe with ice. The difference between average global temperatures today and during those ice ages is only about 9 degrees Fahrenheit (5 degrees Celsius), and the swings have tended to happen slowly, over hundreds of thousands of years.
But with concentrations of greenhouse gases rising, Earth's remaining ice sheets such as Greenland and Antarctica are starting to melt too. That extra water could raise sea levels significantly, and quickly. By 2050, sea levels are predicted to rise between one and 2.3 feet as glaciers melt.
As the mercury rises, the climate can change in unexpected ways. In addition to sea levels rising, weather can become more extreme. This means more intense major storms, more rain followed by longer and drier droughts—a challenge for growing crops—changes in the ranges in which plants and animals can live, and loss of water supplies that have historically come from glaciers.
'''

In [4]:
g2='''
 What is global warming? Over the past 50 years, the average global temperature has increased at the fastest rate in recorded history. And experts see the trend is accelerating: All but one of the 16 hottest years in NASA’s 134-year record have occurred since 2000. Climate change deniers have argued that there has been a “pause” or a “slowdown” in rising global temperatures, but several recent studies, including a 2015 paper published in the journal Science, have disproved this claim. And scientists say that unless we curb global-warming emissions, average U.S. temperatures could increase by up to 10 degrees Fahrenheit over the next century.
What causes global warming? Global warming occurs when carbon dioxide (CO2) and other air pollutants and greenhouse gases collect in the atmosphere and absorb sunlight and solar radiation that have bounced off the earth’s surface. Normally, this radiation would escape into space—but these pollutants, which can last for years to centuries in the atmosphere, trap the heat and cause the planet to get hotter. That's what's known as the greenhouse effect. In the United States, the burning of fossil fuels to make electricity is the largest source of heat-trapping pollution, producing about two billion tons of CO2 every year. Coal-burning power plants are by far the biggest polluters. The country’s second-largest source of carbon pollution is the transportation sector, which generates about 1.7 billion tons of CO2 emissions a year. Curbing dangerous climate change requires very deep cuts in emissions, as well as the use of alternatives to fossil fuels worldwide. The good news is that we’ve started a turnaround: CO2 emissions in the United States actually decreased from 2005 to 2014, thanks in part to new, energy-efficient technology and the use of cleaner fuels. And scientists continue to develop new ways to modernize power plants, generate cleaner electricity, and burn less gasoline while we drive. The challenge is to be sure these solutions are put to use and widely adopted.
How is global warming linked to extreme weather? Scientists agree that the earth’s rising temperatures are fueling longer and hotter heat waves, more frequent droughts, heavier rainfall, and more powerful hurricanes. In 2015, for example, scientists said that an ongoing drought in California—the state’s worst water shortage in 1,200 years—had been intensified by 15 percent to 20 percent by global warming. They also said the odds of similar droughts happening in the future had roughly doubled over the past century. And in 2016, the National Academies of Science, Engineering, and Medicine announced that it’s now possible to confidently attribute certain weather events, like some heat waves, directly to climate change. The earth’s ocean temperatures are getting warmer, too—which means that tropical storms can pick up more energy. So global warming could turn, say, a category 3 storm into a more dangerous category 4 storm. In fact, scientists have found that the frequency of North Atlantic hurricanes has increased since the early 1980s, as well as the number of storms that reach categories 4 and 5. In 2005, Hurricane Katrina—the costliest hurricane in U.S. history—struck New Orleans; the second-costliest, Hurricane Sandy, hit the East Coast in 2012. The impacts of global warming are being felt across the globe. Extreme heat waves have caused tens of thousands of deaths around the world in recent years. And in an alarming sign of events to come, Antarctica has been losing about 134 billion metric tons of ice per year since 2002. This rate could speed up if we keep burning fossil fuels at our current pace, some experts say, causing sea levels to rise several meters over the next 50 to 150 years.

 What are the other effects of global warming? Each year, scientists learn more about the consequences of global warming, and many agree that environmental, economic, and health consequences are likely to occur if current trends continue. Here’s just a smattering of what we can look forward to: Melting glaciers, early snowmelt, and severe droughts will cause more dramatic water shortages and increase the risk of wildfires in the American West. Rising sea levels will lead to coastal flooding on the Eastern Seaboard, especially in Florida, and in other areas such as the Gulf of Mexico. Forests, farms, and cities will face troublesome new pests, heat waves, heavy downpours, and increased flooding. All those factors will damage or destroy agriculture and fisheries. Disruption of habitats such as coral reefs and Alpine meadows could drive many plant and animal species to extinction. Allergies, asthma, and infectious disease outbreaks will become more common due to increased growth of pollen-producing ragweed, higher levels of air pollution, and the spread of conditions favourable to pathogens and mosquitoes.
Where does the United States stand in terms of global-warming contributors? In recent years, China has taken the lead in global-warming pollution, producing about 28 percent of all CO2 emissions. The United States comes in second. Despite making up just 4 percent of the world’s population, we produce a whopping 16 percent of all global CO2 emissions—as much as the European Union and India combined. And America is still number one, by far, in cumulative emissions over the past 150 years. Our responsibility matters to other countries, and it should matter to us, too.
Is the United States doing anything to prevent global warming? The United States have started working. But in order to avoid the worst effects of climate change, we need to do a lot more—together with other countries—to reduce our dependence on fossil fuels and start using clean energy instead. In 2015, the U.S. Environmental Protection Agency pledged to reduce carbon pollution from our power plants by nearly a third by 2030, relative to 2005 levels, through its Clean Power Plan. But fast-forward to 2017, and under the Trump Administration, the EPA proposed repealing this critical tool for curbing climate change. Likewise, while under the Obama administration, the U.S. Department of Transportation proposed carbon pollution and fuel economy standards intended to cut emissions through the 2020s, under Trump administration, the DOT is working to roll back those clean vehicle safeguards that protect the climate and our health. Fortunately, state leaders—including in car country itself—recognize that clean transportation must remain a priority if we are to address the costly risks of climate change and protect public health. And regional efforts around the country are helping to boost the electric car market, which saw an increase in sales for 2017 over 2016. Our clean energy economy is growing too, despite federal efforts to derail it. In 2016, wind employment grew by 32 percent and solar jobs increased by 25 percent. Globally, at the United Nations Conference on Climate Change in Paris, 195 countries—including the United States, at the time—agreed to pollution-cutting provisions with a goal of preventing the average global temperature from rising more than 1.5 degrees Celsius above preindustrial times. (Scientists say we must stay below a two-degree increase to avoid catastrophic climate impacts.) To help make the deal happen, the Obama administration pledged $3 billion to the Green Climate Fund, an international organization dedicated to helping poor countries adopt cleaner energy technologies. Under the terms of

 the Paris agreement, participating nations will meet every five years, starting in 2020, to revise their plans for cutting CO2 emissions. Beginning in 2023, they will also have to publicly report their progress.While in 2017, President Trump announced the country’s withdrawal from the Paris climate agreement and to eliminate “harmful and unnecessary policies such as the Climate Action Plan,” Americans are forging ahead without him.
Through initiatives like the United States Climate Alliance, the Regional Greenhouse Gas Initiative, We Are Still In, and Climate Mayors, state, business, and local leaders have pledged to honor and uphold the goals of the Paris Agreement. More than 25 cities in 17 states, with populations totaling more than 5 million have adopted resolutions that will enable them to get 100 percent of their electricity from renewable sources like wind and solar. Even better, a new initiative by former New York City mayor Michael Bloomberg gives the urban layer of this movement a boost. He’s asked mayors from the 100 most populous cities in the country to share their plans for making their buildings and transportation systems run cleaner and more efficiently. The 20 that show the greatest potential for cutting the dangerous carbon pollution that’s driving climate change will share a total of $70 million in technical assistance funding provided by Bloomberg Philanthropies and partners.
Is global warming too big of a problem for me to help tackle? Wondering how to stop global warming? Reduce your own carbon footprint by following a few easy steps. Make conserving energy a part of your daily routine and your decisions as a consumer. When you shop for new appliances like refrigerators, washers, and dryers, look for products with the government’s Energy Star label; they meet a higher standard for energy efficiency than the minimum federal requirements. When you buy a car, look for one with the highest gas mileage and lowest emissions. You can also reduce your emissions by taking public transportation or carpooling when possible. And while new federal and state standards are a step in the right direction, much more needs to be done. Voice your support of climate-friendly and climate change preparedness policies, and tell your representatives that transitioning from dirty fossil fuels to clean power should be a top priority—because it’s vital to building healthy, more secure communities.
'''

In [5]:
p1 = '''elhi Assembly election campaign is in its last lap. Nearly 1.5 crore voters will vote on February 8 (the coming Saturday) in all 70 assembly constituencies to elect their next government in Delhi.

All three main parties the ruling Aam Aadmi Party (AAP) and challengers the BJP and the Congress have released their manifestos for the Delhi Assembly election. The AAP of Chief Minister Arvind Kejriwal is in the fray with once more slogan.

The BJP is pitching its corruption-free, development oriented election plank in the Delhi Assembly polls. And, the Congress’s catchphrase in the Delhi election is, We want Congress waali Dilli.

Let us have a look at what all three parties have offered as election promise to the people of Delhi. First, the ruling AAP of Kejriwal.

BIJLI-PAANI

In mid-January, Arvind Kejriwal released a 10-point poll pledge on his personal guarantee saying if elected, the next AAP government will deliver on these promises. Kejriwal promised to provide uninterrupted power and piped water supply to all households.

In his election campaigns, Kejriwal is already claiming that his government has ensured 24x7 power supply in Delhi in the past five years. Connecting every house with clean piped water is his promise for the next five years.

Like the AAP, the BJP has also promised water supply in every home of Delhi.

Against the AAP's free power-free water campaign, the Congress too promises free electricity up to 300 units each household. The Kejriwal government has kept it free up to 200 units.

UNAUTHORISED COLONIES

The AAP promises to provide basic facilities for unauthorised colonies, affordable and quality healthcare and efficient garbage disposal for better cleanliness in Delhi.

ADVERTISEMENT

In the run up to Delhi election, the BJP-led central government decided to regularise 1,797 unauthorised colonies in Delhi. The BJP in its manifesto has said a colony development board will be set up for development of these colonies to benefit more than 50 lakh people.

The Congress has promised to regularise rest of the unauthorised colonies in Delhi and launch a project worth Rs 35,000 crore for their uplift.

YOUTH AND WOMEN

Good education for students until graduation is a promise that appears as extension of Kejriwal government’s school education programme.

The BJP promises to build 10 new colleges and 200 new schools in Delhi besides one library each in 280 wards of Delhi.

Kejriwal has promised to deploy Mohalla Marshals for improved security for women. A similar promise of deployment of marshals in public buses was made by the AAP in 2015 but it remained largely unfulfilled. This promise was made in the backdrop of the Nirbhaya gangrape incident in which six people had brutalised a paramedic student who later died.


All three parties have tried to woo Delhi's young voters in their poll promises.
The BJP, which has not projected its chief ministerial candidate in Delhi election, has promised to provide 10 lakh jobs in Delhi over the next five years. This is supposed to be in addition to the vacant posts that the BJP has promised to fill all vacant posts in Delhi.

It has promised to distribute electric scooters among college going girls, and bicycles for girls in Class 9th. This is on the lines of populist policies implemented in states like Bihar where the government distributed among school going girls. The BJP is part of the Nitish Kumar government in Bihar and contesting the Delhi election in alliance with his Janata Dal (United).

The BJP has promised to give financial assistance of Rs 51,000 for marriage of girls of economically weaker sections. The party has also promised to give wheat flour at Rs 2 per kg to BPL families. A similar scheme in Chhattisgarh sustained the BJP government in power for 15 years.

Inspired by Congress’s Lok Sabha poll plank Nyay (Nyuntam Aay Yojana), the party has promised to launch a Yuvaa Swabhiman Yojana in Delhi offering monthly unemployment allowance of Rs 5,000 to graduates and Rs 7,500 to postgraduates.


Delhi Election Commission is running a voters' awareness campaign for greater participation in Delhi Assembly election on February 8. (Photo: Twitter/@CeodelhiOffice)
COMMUTING IN DELHI

Kejriwal’s guarantee card talks about biggest and cheapest public transport in Delhi in a fleet of 11,000 buses and 500 km Delhi Metro network. This is significant in the view that he has faced flak for not expanding the Delhi bus fleet which Kejriwal blamed on a court case.

The BJP has promised to invest massively in improving road network with projects worth Rs 55,000 crore in Delhi. Another Rs 10,000 crore will be spent, the BJP says, for infrastructure development in Delhi.

ADVERTISEMENT

The Congress has promised to immediately procure 15,000 electric buses to increase the fleet size of Delhi Transport Corporation (DTC).

Kejriwal has promised free travel on public transport to students in Delhi. This is currently available to women currently. The Congress has also promised subsidised metro travel for women, students and elderly people.

AIR POLLUTION. polls.

With air pollution being the biggest worry of Delhi residents, Kejriwaal has promised to bring air pollution by one-third of the existing level by planting 2 crore trees and cleaning Yamuna river.

On pollution, Delhi BJP chief Manoj Tiwari said if elected to power, the BJP will solve air pollution problem in two years.

On the other hand, the Congress has promised to spend 25 per cent of Delhi’s annual budget to combat the air pollution problem and improve transportation.


An over 100-year old Delhi voter at polling booth during Lok Sabha election 2019. The Election Commission made arrangements for pickup and drop for this voter identified as Kitabunnisa by the poll body in a Twitter post.
OTHER ISSUES IN POLL PROMISES

The BJP has promised to stop sealing in Delhi. Sealing is carried in Delhi in accordance with court orders in cases where public properties were encroached upon by residents and business entities.

Among other promises, the BJP said it will implement Ayushman Bharat Scheme, housing for all scheme and PM Kisan scheme in Delhi. Aushman Bharat has been a sticking point between the Kejriwal government and the Modi government. Kejriwal says the Delhi government’s health scheme is all-inclusive while Ayushman Bharat is selective in nature.

Interestingly, the Congress has promised passing a resolution against the Citizenship Amendment Act in its first assembly session if the party is voted to power. The Congress’s state governments in Rajasthan and Punjab have passed similar resolutions.

A POLL OF HOPES FOR CHALLENGERS

The BJP has remained the principal challenger in every Delhi Assembly election since 1998. The AAP suddenly appeared on the Delhi election scene in 2013 forming a minority government and stormed to power in 2015 but the standing of the BJP did not change as the principal challenger in Delhi politics. Earlier, it was competing with the Congress for power, now Kejriwal’s AAP has taken that place.

In its election manifesto, which the BJP presents as vision document, the party promises to provide a corruption-free and transparent government with a focus on development in Delhi if the party is elected to power.

The Congress too is in race in Delhi election. It is hardselling the achievements of three successive Sheila Dikshit governments in Delhi from 1998 to 2013 with its theme song Congress waali Dilli while Kejriwal sounds confident that there is strong pro-incumbency in Delhi. We will find out who stands where in Delhi election on February 11, the day of counting of votes and results.'''

In [6]:
text_data = []

for line in (g1,g2,p1):
    tokens = prepare_text_for_lda(line)
    text_data.append(tokens)

In [7]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [51]:
NUM_TOPICS = 50
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=50)
ldamodel.save('model5.gensim')

In [52]:
import re
topics = ldamodel.print_topics(num_words=1)
# for topic in topics:
#     print(topic)
#     print('')
#     print(re.findall(r'"(.*?)"', topic[1]))
#     print('\n')

In [53]:
len(topics)

20

In [54]:
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type complex is not JSON serializable

In [10]:
new_doc = '''what is carbon dioxide'''
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

for item in new_doc_bow:
    print(dictionary.get(item[0]), item[1])

[(22, 1)]
[(0, 0.010000524), (1, 0.010000524), (2, 0.010000524), (3, 0.010000524), (4, 0.010000524), (5, 0.010000524), (6, 0.010000524), (7, 0.010000524), (8, 0.010000524), (9, 0.010000524), (10, 0.010000524), (11, 0.010000524), (12, 0.010000524), (13, 0.010000524), (14, 0.010000524), (15, 0.010000524), (16, 0.010000524), (17, 0.010000524), (18, 0.010000524), (19, 0.010000524), (20, 0.010000524), (21, 0.010000524), (22, 0.010000524), (23, 0.010000524), (24, 0.010000524), (25, 0.010000524), (26, 0.010000524), (27, 0.010000524), (28, 0.010000524), (29, 0.010000524), (30, 0.010000524), (31, 0.010000524), (32, 0.010000524), (33, 0.010000524), (34, 0.010000524), (35, 0.010000524), (36, 0.010000524), (37, 0.010000524), (38, 0.010000524), (39, 0.010000524), (40, 0.010000524), (41, 0.010000524), (42, 0.010000524), (43, 0.010000524), (44, 0.010000524), (45, 0.010000524), (46, 0.5099743), (47, 0.010000524), (48, 0.010000524), (49, 0.010000524)]
carbon dioxide 1


In [11]:
# import pyLDAvis.gensim
# dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
# corpus = pickle.load(open('corpus.pkl', 'rb'))
# lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
# lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
# pyLDAvis.display(lda_display)

In [12]:
new_doc = '''What Is Air Pollution?
Air pollution refers to the release of pollutants into the air that are detrimental to human health and the planet as a whole.

The Clean Air Act authorizes the U.S. Environmental Protection Agency (EPA) to protect public health by regulating the emissions of these harmful air pollutants. The NRDC has been a leading authority on this law since it was established in 1970.

What Causes Air Pollution?
“Most air pollution comes from energy use and production,” says John Walke, director of the Clean Air Project, part of the Climate and Clean Air program at NRDC. “Burning fossil fuels releases gases and chemicals into the air.” And in an especially destructive feedback loop, air pollution not only contributes to climate change but is also exacerbated by it. “Air pollution in the form of carbon dioxide and methane raises the earth’s temperature,” Walke says. “Another type of air pollution is then worsened by that increased heat: Smog forms when the weather is warmer and there’s more ultraviolet radiation.” Climate change also increases the production of allergenic air pollutants including mold (thanks to damp conditions caused by extreme weather and increased flooding) and pollen (due to a longer pollen season and more pollen production).

Effects of Air Pollution
“While we’ve made progress over the last 40-plus years improving air quality in the U.S. thanks to the Clean Air Act, climate change will make it harder in the future to meet pollution standards, which are designed to protect health,” says Kim Knowlton, senior scientist and deputy director of the NRDC Science Center.

Smog and soot
These two are the most prevalent types of air pollution. Smog, or “ground-level ozone,” as it is more wonkily called, occurs when emissions from combusting fossil fuels react with sunlight. Soot, or “particulate matter,” is made up of tiny particles of chemicals, soil, smoke, dust, or allergens, in the form of gas or solids, that are carried in the air. The EPA’s “Plain English Guide to the Clean Air Act” states, “In many parts of the United States, pollution has reduced the distance and clarity of what we see by 70 percent.” The sources of smog and soot are similar. “Both come from cars and trucks, factories, power plants, incinerators, engines—anything that combusts fossil fuels such as coal, gas, or natural gas,” Walke says. The tiniest airborne particles in soot—whether they’re in the form of gas or solids—are especially dangerous because they can penetrate the lungs and bloodstream and worsen bronchitis, lead to heart attacks, and even hasten death.

Smog can irritate the eyes and throat and also damage the lungs—especially of people who work or exercise outside, children, and senior citizens. It’s even worse for people who have asthma or allergies—these extra pollutants only intensify their symptoms and can trigger asthma attacks.

Hazardous air pollutants
These are either deadly or have severe health risks even in small amounts. Almost 200 are regulated by law; some of the most common are mercury, lead, dioxins, and benzene. “These are also most often emitted during gas or coal combustion, incinerating, or in the case of benzene, found in gasoline,” Walke says. Benzene, classified as a carcinogen by the EPA, can cause eye, skin, and lung irritation in the short term and blood disorders in the long term. Dioxins, more typically found in food but also present in small amounts in the air, can affect the liver in the short term and harm the immune, nervous, and endocrine systems, as well as reproductive functions. Lead in large amounts can damage children’s brains and kidneys, and even in small amounts it can affect children’s IQ and ability to learn. Mercury affects the central nervous system.

Polycyclic aromatic hydrocarbons, or PAHs, are toxic components of traffic exhaust and wildfire smoke. In large amounts, they have been linked to eye and lung irritation, blood and liver issues, and even cancer. In one recent study, the children of mothers who’d had higher PAH exposure during pregnancy had slower brain processing speeds and worse symptoms of ADHD.

Greenhouse gases
By trapping the earth’s heat in the atmosphere, greenhouse gases lead to warmer temperatures and all the hallmarks of climate change: rising sea levels, more extreme weather, heat-related deaths, and increasing transmission of infectious diseases like Lyme. According to a 2014 EPA study, carbon dioxide was responsible for 81 percent of the country’s total greenhouse gas emissions, and methane made up 11 percent. “Carbon dioxide comes from combusting fossil fuels, and methane comes from natural and industrial sources, including the large amounts that are released during oil and gas drilling,” Walke says. “We emit far larger amounts of carbon dioxide, but methane is significantly more potent, so it’s also very destructive.” Another class of greenhouse gases, hydrofluorocarbons (HFCs), are thousands of times more powerful than carbon dioxide in their ability to trap heat. In October 2016, more than 140 countries reached an agreement to reduce the use of these chemicals—which are used in air conditioners and refrigerators—and find greener alternatives over time. David Doniger, director of NRDC’s Climate and Clean Air program, writes, “NRDC estimates that the agreed HFC phase-down will avoid the equivalent of more than 80 billion tons of CO2 over the next 35 years.”

Pollen and mold
Mold and allergens from trees, weeds, and grass are also carried in the air, are exacerbated by climate change, and can be hazardous to health. They are not regulated by the government and are less directly connected to human actions, but they can be considered air pollution. “When homes, schools, or businesses get water damage, mold can grow and can produce allergenic airborne pollutants,” Knowlton says. “Mold exposure can precipitate asthma attacks or an allergic response, and some molds can even produce toxins that would be dangerous for anyone to inhale.”

Pollen allergies are worsening because of climate change. “Lab and field studies are showing that the more carbon dioxide pollen-producing plants—especially ragweed—are grown in, the bigger they grow and the more pollen they produce,” Knowlton says. “Climate change also extends the pollen production season, and some studies are beginning to suggest that ragweed pollen itself might be becoming a more potent allergen.” That means more people will suffer runny noses, fevers, itchy eyes, and other symptoms.

How to Help Reduce Air Pollution
“The less gasoline we burn, the better we’re doing to reduce air pollution and harmful effects of climate change,” Walke says. “Make good choices about transportation. When you can, walk, ride a bike, or take public transportation. For driving, choose cars that get better miles per gallon of gas or choose an electric car.” You can also investigate your power provider options—you may be able to request that your electricity be supplied by wind or solar. Buying your food locally cuts down on the fossil fuels burned in trucking or flying food in from across the country. And perhaps most important, “Support leaders who push for clean air and water and responsible steps on climate change,” Walke says.

How to Protect Your Health
“When you see in the newspaper or hear on the weather report that pollution levels are high, it may be useful to limit the time when children go outside or you go for a jog,” Walke says. Generally, ozone levels tend to be lower in the morning.
When you do exercise outside, stay as far as you can from heavily trafficked roads. Then shower and wash your clothes to remove fine particles.
If the air quality is bad, stay inside with windows closed.
Wear sunscreen. When ultraviolet radiation comes through the weakened ozone layer, it can cause skin damage and skin cancer.'''
text_data = []

tokens = prepare_text_for_lda(new_doc)
text_data.append(tokens)
    
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
NUM_TOPICS = 1
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=50)

topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)
    print('')
    print(re.findall(r'"(.*?)"', topic[1]))
    print('\n')

(0, '0.023*"air pollution" + 0.019*"climate change" + 0.017*"walke" + 0.013*"the air" + 0.013*"carbon dioxide" + 0.010*"methane" + 0.010*"fossil fuels" + 0.008*"greenhouse gases" + 0.008*"the form" + 0.008*"benzene"')

['air pollution', 'climate change', 'walke', 'the air', 'carbon dioxide', 'methane', 'fossil fuels', 'greenhouse gases', 'the form', 'benzene']




In [13]:
len(topics)

1